# Mobile net

In [ ]:

'''
MobileNet in TensorFlow2.

Reference:
[1] Howard, Andrew G., et al.
    "Mobilenets: Efficient convolutional neural networks for mobile vision applications."
    arXiv preprint arXiv:1704.04861 (2017).
'''
import tensorflow as tf
from tensorflow.keras import layers



class Block(tf.keras.Model):
    '''Depthwise convolution + pointwise convolution'''
    def __init__(self, in_channels, out_channels, strides=1):
        super(Block, self).__init__()
        self.conv1 = layers.Conv2D(in_channels, kernel_size=3, strides=strides, padding='same',
                                   groups=in_channels, use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=1, use_bias=False)
        self.bn2 = layers.BatchNormalization()

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = tf.keras.activations.relu(self.bn2(self.conv2(out)))
        return out

class MobileNet(tf.keras.Model):
    # (128, 2) represents convolution layer with filters=128, strides=2
    config = [64, (128,2), 128, (256,2), 256, (512,2), 512, 512, 512, 512, 512, (1024,2), 1024]

    def __init__(self, num_classes, input_shape=(32, 32, 3)):
        super(MobileNet, self).__init__()
        self.conv1 = layers.Conv2D(32, kernel_size=3, padding='same', use_bias=False, input_shape=input_shape)
        self.bn1 = layers.BatchNormalization()
        self.layer = self._make_layers(in_channels=32)
        self.avg_pool2d = layers.AveragePooling2D(pool_size=2)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.layer(out)
        out = self.avg_pool2d(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

    def _make_layers(self, in_channels):
        layer = []
        for c in self.config:
            out_channels = c if isinstance(c, int) else c[0]
            strides = 1 if isinstance(c, int) else c[1]
            layer += [Block(in_channels, out_channels, strides)]
            in_channels = out_channels
        return tf.keras.Sequential(layer)

# Main


In [ ]:
# Tải dữ liệu CIFAR10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Chuẩn hóa dữ liệu
x_train = x_train / 255.0
x_test = x_test / 255.0


170498071/170498071 [==============================] - 5s 0us/step


In [ ]:
# Tạo và huấn luyện mô hình
model = MobileNet(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train, epochs=10)

Epoch 1/10
1563/1563 [==============================] - 73s 28ms/step - loss: 1.6572 - accuracy: 0.3927
Epoch 2/10
1563/1563 [==============================] - 38s 25ms/step - loss: 1.2565 - accuracy: 0.5545
Epoch 3/10
1563/1563 [==============================] - 39s 25ms/step - loss: 1.0128 - accuracy: 0.6474
Epoch 4/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.8405 - accuracy: 0.7102
Epoch 5/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.7148 - accuracy: 0.7526
Epoch 6/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.6262 - accuracy: 0.7835
Epoch 7/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.5362 - accuracy: 0.8139
Epoch 8/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.4592 - accuracy: 0.8410
Epoch 9/10
1563/1563 [==============================] - 39s 25ms/step - loss: 0.3922 - accuracy: 0.8646
Epoch 10/10
1563/1563 [==============================] - 39s 25m

In [ ]:
# Đánh giá mô hình
model.evaluate(x_test, y_test)

313/313 [==============================] - 6s 12ms/step - loss: 0.6717 - accuracy: 0.7890


[0.6717327237129211, 0.7889999747276306]

In [ ]:
model.save("model_mobilenet")

# Senet model

In [ ]:
'''
SEResNet18/34/50/101/152 in TensorFlow2.
SEPreActResNet18/34/50/101/152 in TensorFlow2.

Reference:
[1] Hu, Jie, Li Shen, and Gang Sun.
    "Squeeze-and-excitation networks."
    Proceedings of the IEEE conference on computer vision and pattern recognition. 2018.
'''
import tensorflow as tf
from tensorflow.keras import layers
!pip install tensorflow-addons
import tensorflow_addons as tfa
import sys

class SELayer(tf.keras.Model):
    def __init__(self, out_channels, reduction=16):
        super(SELayer, self).__init__()
        self.adapt_pool2d = tfa.layers.AdaptiveAveragePooling2D(output_size=1)
        self.squeeze = tf.keras.Sequential([
            layers.Conv2D(out_channels//reduction, kernel_size=1, use_bias=False, activation='relu'),
            layers.Conv2D(out_channels, kernel_size=1, use_bias=False, activation='sigmoid')
        ])

    def call(self, x):
        # Squeeze
        out = self.adapt_pool2d(x)
        out = self.squeeze(out)
        ###old code
        # Excitation
        # out = x * tf.broadcast_to(out, x.shape)
        # return out
        #####
        # Excitation
        shape = tf.shape(x)
        out = tf.broadcast_to(out, shape)
        out = x * out
        return out

class BasicBlock(tf.keras.Model):
    expansion = 1

    def __init__(self, in_channels, out_channels, strides=1, reduction=16):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=strides, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.se = SELayer(self.expansion*out_channels, reduction)

        if strides != 1 or in_channels != self.expansion*out_channels:
            self.shortcut = tf.keras.Sequential([
                layers.Conv2D(self.expansion*out_channels, kernel_size=1, strides=strides, use_bias=False),
                layers.BatchNormalization()
            ])
        else:
            self.shortcut = lambda x: x

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.se(self.bn2(self.conv2(out)))
        out = layers.add([self.shortcut(x), out])
        out = tf.keras.activations.relu(out)
        return out

class BuildSEResNet(tf.keras.Model):
    def __init__(self, block, num_blocks, num_classes, reduction=16,input_shape=(32, 32, 3)):
        super(BuildSEResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = layers.Conv2D(64, kernel_size=3, strides=1, padding='same', use_bias=False, input_shape=input_shape)
        self.bn1 = layers.BatchNormalization()
        self.layer1 = self._make_layer(block, 64, num_blocks[0], strides=1, reduction=reduction)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], strides=2, reduction=reduction)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], strides=2, reduction=reduction)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], strides=2, reduction=reduction)
        self.avg_pool2d = layers.AveragePooling2D(pool_size=4)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(num_classes, activation='softmax')

    def call(self, x):
        out = tf.keras.activations.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool2d(out)
        out = self.flatten(out)
        out = self.fc(out)
        return out

    def _make_layer(self, block, out_channels, num_blocks, strides, reduction):
        stride = [strides] + [1]*(num_blocks-1)
        layer = []
        for s in stride:
            layer += [block(self.in_channels, out_channels, s, reduction)]
            self.in_channels = out_channels * block.expansion
        return tf.keras.Sequential(layer)

def SEResNet(num_classes):
  return  BuildSEResNet(BasicBlock, [2, 2, 2, 2], num_classes)


In [ ]:
# Tải dữ liệu CIFAR10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Chuẩn hóa dữ liệu
x_train = x_train / 255.0
x_test = x_test / 255.0

In [ ]:
# Tạo và huấn luyện mô hình
model_SENet = BuildSEResNet(BasicBlock, [2, 2, 2, 2], 10)

model_SENet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_SENet.fit(x_train, y_train, epochs=5)


Epoch 1/5
1563/1563 [==============================] - 105s 46ms/step - loss: 1.1898 - accuracy: 0.5749
Epoch 2/5
1563/1563 [==============================] - 71s 45ms/step - loss: 0.7205 - accuracy: 0.7479
Epoch 3/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.5360 - accuracy: 0.8133
Epoch 4/5
1563/1563 [==============================] - 71s 45ms/step - loss: 0.4042 - accuracy: 0.8596
Epoch 5/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.2960 - accuracy: 0.8956


In [ ]:
model_SENet.summary()

Model: "build_se_res_net_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_316 (Conv2D)         multiple                  1728      
                                                                 
 batch_normalization_172 (B  multiple                  256       
 atchNormalization)                                              
                                                                 
 sequential_123 (Sequential  (None, 32, 32, 64)        149504    
 )                                                               
                                                                 
 sequential_127 (Sequential  (None, 16, 16, 128)       530944    
 )                                                               
                                                                 
 sequential_131 (Sequential  (None, 8, 8, 256)         2118656   
 )                                              

In [ ]:
model_SENet.evaluate(x_test, y_test)

313/313 [==============================] - 7s 17ms/step - loss: 0.5439 - accuracy: 0.8167


[0.543856680393219, 0.8166999816894531]

In [ ]:
model_SENet.fit(x_train, y_train, epochs=5)

Epoch 1/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.2036 - accuracy: 0.9297
Epoch 2/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.1448 - accuracy: 0.9494
Epoch 3/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.1055 - accuracy: 0.9624
Epoch 4/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.0878 - accuracy: 0.9701
Epoch 5/5
1563/1563 [==============================] - 70s 45ms/step - loss: 0.0707 - accuracy: 0.9750


In [ ]:
model_SENet.evaluate(x_test, y_test)

313/313 [==============================] - 4s 14ms/step - loss: 0.7341 - accuracy: 0.8351


[0.7341488003730774, 0.835099995136261]